## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-21-13-11-24 +0000,nypost,Mamdani’s DSA pals launch Bad Bunny-themed ad ...,https://nypost.com/2026/02/21/us-news/mamdanis...
1,2026-02-21-13-10-56 +0000,nypost,NYC cops being harassed and doxxed by social m...,https://nypost.com/2026/02/21/us-news/nyc-cops...
2,2026-02-21-13-00-00 +0000,wsj,Missed Funerals and Blocked Toilets: Iran Depl...,https://www.wsj.com/us-news/missed-funerals-an...
3,2026-02-21-12-53-45 +0000,nyt,Ferreira completes his halfpipe set with gold.,https://www.nytimes.com/2026/02/20/world/alex-...
4,2026-02-21-12-50-50 +0000,nypost,Woman shoved onto NYC subway tracks by crazed ...,https://nypost.com/2026/02/21/us-news/woman-sh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2514/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
109,trump,77
113,court,48
112,supreme,44
152,tariffs,37
111,tariff,31


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,343
39,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,269
197,2026-02-20-20-56-39 +0000,wapo,"Trump imposes new tariffs, denounces Supreme C...",https://www.washingtonpost.com/national-securi...,266
150,2026-02-20-23-26-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,263
306,2026-02-20-15-03-53 +0000,cbc,Trump imposes new 10% global tariff after U.S....,https://www.cbc.ca/news/world/livestory/scotus...,258


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,343,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
306,84,2026-02-20-15-03-53 +0000,cbc,Trump imposes new 10% global tariff after U.S....,https://www.cbc.ca/news/world/livestory/scotus...
133,64,2026-02-21-00-49-41 +0000,nypost,Los Angeles small business owner takes wait-an...,https://nypost.com/2026/02/20/business/los-ang...
279,47,2026-02-20-16-34-59 +0000,nyt,Trump says he’s considering a limited strike o...,https://www.nytimes.com/live/2026/02/20/us/tru...
323,42,2026-02-20-14-01-00 +0000,wsj,Key inflation metrics tracked by the Fed accel...,https://www.wsj.com/economy/central-banking/fe...
290,40,2026-02-20-15-58-01 +0000,nypost,The government agency relying on Wall Street t...,https://nypost.com/2026/02/20/business/develop...
219,32,2026-02-20-19-56-29 +0000,nyt,Former Prince Andrew’s Arrest Upends Royal Fam...,https://www.nytimes.com/2026/02/20/world/europ...
296,31,2026-02-20-15-40-27 +0000,48hills,Screen Grabs: End of days at Fear and Faith Ho...,https://48hills.org/2026/02/screen-grabs-end-o...
319,29,2026-02-20-14-34-29 +0000,nypost,NFL commish thought Bad Bunny’s halftime show ...,https://nypost.com/2026/02/20/business/nfls-ro...
74,29,2026-02-21-05-55-03 +0000,nypost,Viral video shows anti-ICE school walkout teen...,https://nypost.com/2026/02/21/us-news/ohio-hig...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
